<div style="text-align: center; background-color:rgb(32, 5, 129); padding: 10px; border-radius: 10px;">
    <span style="color: white; font-size: 40px; font-weight: bold;">Preprocissing Data To build Model</span>
</div>

In [1]:
import pandas as pd
import numpy as np
import scipy as sc
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import joblib
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
import time
import pandas as pd
from datetime import datetime
import holidays
from dateutil.relativedelta import relativedelta, FR
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import RFE
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler


In [2]:
carSales =pd.read_csv('../../Milestone1/Data/Car_sales_CleanData.csv')
carSales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23090 entries, 0 to 23089
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Date           23090 non-null  object
 1   Customer Name  23090 non-null  object
 2   Gender         23090 non-null  object
 3   Annual Income  23090 non-null  int64 
 4   Dealer_Name    23090 non-null  object
 5   Company        23090 non-null  object
 6   Model          23090 non-null  object
 7   Engine         23090 non-null  object
 8   Transmission   23090 non-null  object
 9   Color          23090 non-null  object
 10  Price ($)      23090 non-null  int64 
 11  Dealer_No      23090 non-null  object
 12  Body Style     23090 non-null  object
 13  Phone          23090 non-null  int64 
 14  Dealer_Region  23090 non-null  object
 15  Year           23090 non-null  int64 
 16  Month          23090 non-null  int64 
 17  DayOfWeek      23090 non-null  int64 
 18  Season         23090 non-n

In [3]:
carSales = carSales.drop(columns=['Customer Name', 'Dealer_Name', 'Dealer_No ', 'Phone'])

In [4]:
carSales.info()
carSales.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23090 entries, 0 to 23089
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Date           23090 non-null  object
 1   Gender         23090 non-null  object
 2   Annual Income  23090 non-null  int64 
 3   Company        23090 non-null  object
 4   Model          23090 non-null  object
 5   Engine         23090 non-null  object
 6   Transmission   23090 non-null  object
 7   Color          23090 non-null  object
 8   Price ($)      23090 non-null  int64 
 9   Body Style     23090 non-null  object
 10  Dealer_Region  23090 non-null  object
 11  Year           23090 non-null  int64 
 12  Month          23090 non-null  int64 
 13  DayOfWeek      23090 non-null  int64 
 14  Season         23090 non-null  object
dtypes: int64(5), object(10)
memory usage: 2.6+ MB


(23090, 15)

# Feature Engineering

In [5]:
# Egyptian holidays in English
eg_holidays = holidays.Egypt(language='en')

# Function to get Black Friday (last Friday in November)
def get_black_friday(year):
    last_day = datetime(year, 11, 30)
    return last_day + relativedelta(weekday=FR(-1))

# Function to return fixed custom holidays
def get_custom_holidays(year):
    return {
        "Christmas": datetime(year, 12, 25),
        "Valentine's Day": datetime(year, 2, 14),
        "Mother's Day": datetime(year, 3, 21),
        "Black Friday": get_black_friday(year)
    }

# Function to return the name of the holiday (either from holidays package or custom)
def get_holiday_name(date):
    date = pd.to_datetime(date).date()
    
    # Check official holidays
    if date in eg_holidays:
        return eg_holidays.get(date)

    # Check custom holidays
    for name, d in get_custom_holidays(date.year).items():
        if d.date() == date:
            return name

    return None


In [6]:
carSales['Price_to_Income'] = carSales['Price ($)'] / carSales['Annual Income']

In [7]:
carSales['Holiday'] = carSales['Date'].apply(get_holiday_name)

In [8]:
carSales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23090 entries, 0 to 23089
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             23090 non-null  object 
 1   Gender           23090 non-null  object 
 2   Annual Income    23090 non-null  int64  
 3   Company          23090 non-null  object 
 4   Model            23090 non-null  object 
 5   Engine           23090 non-null  object 
 6   Transmission     23090 non-null  object 
 7   Color            23090 non-null  object 
 8   Price ($)        23090 non-null  int64  
 9   Body Style       23090 non-null  object 
 10  Dealer_Region    23090 non-null  object 
 11  Year             23090 non-null  int64  
 12  Month            23090 non-null  int64  
 13  DayOfWeek        23090 non-null  int64  
 14  Season           23090 non-null  object 
 15  Price_to_Income  23090 non-null  float64
 16  Holiday          1248 non-null   object 
dtypes: float64(1

In [9]:
carSales.Holiday.value_counts()

Holiday
Christmas                                     169
(تقدير) عطلة عيد الفطر                        167
Black Friday                                  139
(تقدير) عيد الأضحى                            113
(تقدير) عطلة عيد الأضحى                        67
عيد ثورة 23 يوليو                              65
عيد العمال                                     63
(تقدير) رأس السنة الهجرية                      63
Valentine's Day                                59
Mother's Day                                   45
عيد الفصح القبطي                               40
(تقدير) عيد المولد النبوي                      39
(تقدير) عيد الفطر                              38
شم النسيم; عيد تحرير سيناء                     35
عيد ثورة 25 يناير                              30
شم النسيم                                      30
عيد القوات المسلحة                             23
(تقدير) يوم عرفة                               20
عيد الميلاد المجيد (تقويم قبطي)                15
عيد تحرير سيناء                           

In [10]:
carSales['Is_Holiday'] = carSales['Date'].apply(lambda x: 1 if get_holiday_name(x) else 0)

In [11]:
carSales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23090 entries, 0 to 23089
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             23090 non-null  object 
 1   Gender           23090 non-null  object 
 2   Annual Income    23090 non-null  int64  
 3   Company          23090 non-null  object 
 4   Model            23090 non-null  object 
 5   Engine           23090 non-null  object 
 6   Transmission     23090 non-null  object 
 7   Color            23090 non-null  object 
 8   Price ($)        23090 non-null  int64  
 9   Body Style       23090 non-null  object 
 10  Dealer_Region    23090 non-null  object 
 11  Year             23090 non-null  int64  
 12  Month            23090 non-null  int64  
 13  DayOfWeek        23090 non-null  int64  
 14  Season           23090 non-null  object 
 15  Price_to_Income  23090 non-null  float64
 16  Holiday          1248 non-null   object 
 17  Is_Holiday  

In [12]:
carSales.dtypes

Date                object
Gender              object
Annual Income        int64
Company             object
Model               object
Engine              object
Transmission        object
Color               object
Price ($)            int64
Body Style          object
Dealer_Region       object
Year                 int64
Month                int64
DayOfWeek            int64
Season              object
Price_to_Income    float64
Holiday             object
Is_Holiday           int64
dtype: object

# **duplicates**

In [13]:
def check_duplicates(df):
    if df.duplicated().sum() > 0:
        print(f"Duplicate rows found: {df.duplicated().sum()}")
        return True
    return False

In [14]:
carSales = carSales.drop_duplicates()
check_duplicates(carSales)

False

# **data types**

In [15]:
def check_data_types(df):
    for col in df.columns:
        if df[col].dtype not in ['int64', 'float64']:
            print(f"Column '{col}' has data type '{df[col].dtype}'")
            return True
    return False
check_data_types(carSales)

Column 'Date' has data type 'object'


True

In [16]:
# Convert all boolean columns to 0 and 1
bool_columns = carSales.select_dtypes(include=['bool']).columns  # Get all bool columns
carSales[bool_columns] = carSales[bool_columns].astype(int)

In [17]:
# Convert 'Date' to datetime
carSales['Date'] = pd.to_datetime(carSales['Date'])



In [18]:
carSales['Year'] = pd.to_datetime(carSales['Date']).dt.year
carSales['Month'] = pd.to_datetime(carSales['Date']).dt.month
carSales['Day'] = pd.to_datetime(carSales['Date']).dt.day
carSales['DayOfWeek'] = pd.to_datetime(carSales['Date']).dt.dayofweek
carSales['WeekOfYear'] = carSales['Date'].dt.isocalendar().week
carSales['Is_Weekend'] = carSales['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)


In [19]:
check_data_types(carSales)

Column 'Date' has data type 'datetime64[ns]'


True

In [20]:
carSales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23089 entries, 0 to 23089
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             23089 non-null  datetime64[ns]
 1   Gender           23089 non-null  object        
 2   Annual Income    23089 non-null  int64         
 3   Company          23089 non-null  object        
 4   Model            23089 non-null  object        
 5   Engine           23089 non-null  object        
 6   Transmission     23089 non-null  object        
 7   Color            23089 non-null  object        
 8   Price ($)        23089 non-null  int64         
 9   Body Style       23089 non-null  object        
 10  Dealer_Region    23089 non-null  object        
 11  Year             23089 non-null  int32         
 12  Month            23089 non-null  int32         
 13  DayOfWeek        23089 non-null  int32         
 14  Season           23089 non-null  object    

# **missing values**

In [21]:
def check_missing_values(df):
    for col in df.columns:
        if df[col].isnull().sum() > 0:
            print(f"Missing values in column '{col}': {df[col].isnull().sum()}")
            return True
    return False
check_missing_values(carSales)

Missing values in column 'Holiday': 21841


True

In [22]:
carSales['Holiday'].fillna('No Holiday', inplace=True)
print(carSales['Holiday'].value_counts())

Holiday
No Holiday                                    21841
Christmas                                       169
(تقدير) عطلة عيد الفطر                          167
Black Friday                                    139
(تقدير) عيد الأضحى                              113
(تقدير) عطلة عيد الأضحى                          67
عيد ثورة 23 يوليو                                65
عيد العمال                                       63
(تقدير) رأس السنة الهجرية                        63
Valentine's Day                                  59
Mother's Day                                     45
عيد الفصح القبطي                                 40
(تقدير) عيد المولد النبوي                        39
(تقدير) عيد الفطر                                38
شم النسيم; عيد تحرير سيناء                       35
عيد ثورة 25 يناير                                30
شم النسيم                                        30
عيد القوات المسلحة                               23
(تقدير) يوم عرفة                                 20
عيد 

C:\Users\DELL\AppData\Local\Temp\ipykernel_16756\1925827676.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  carSales['Holiday'].fillna('No Holiday', inplace=True)


In [23]:
check_missing_values(carSales)

False

# **encoding**

In [24]:
def show_unique_values(df):
    for col in df.select_dtypes(include=['object', 'category']).columns:
        print(f"Column: {col}")
        print(f"Unique Count: {df[col].nunique()}")
        print(f"Unique Percentage: {(df[col].nunique() / len(df)) * 100}%")
        print("\n")
show_unique_values(carSales)


Column: Gender
Unique Count: 2
Unique Percentage: 0.00866213348347698%


Column: Company
Unique Count: 30
Unique Percentage: 0.1299320022521547%


Column: Model
Unique Count: 154
Unique Percentage: 0.6669842782277274%


Column: Engine
Unique Count: 2
Unique Percentage: 0.00866213348347698%


Column: Transmission
Unique Count: 2
Unique Percentage: 0.00866213348347698%


Column: Color
Unique Count: 3
Unique Percentage: 0.012993200225215471%


Column: Body Style
Unique Count: 5
Unique Percentage: 0.02165533370869245%


Column: Dealer_Region
Unique Count: 7
Unique Percentage: 0.03031746719216943%


Column: Season
Unique Count: 4
Unique Percentage: 0.01732426696695396%


Column: Holiday
Unique Count: 22
Unique Percentage: 0.09528346831824679%




In [25]:

# Initialize LabelEncoder
label_encoder = LabelEncoder()
# Iterate over all object columns and apply label encoding
for column in ['Engine', 'Transmission', 'Gender', 'Color', 'Dealer_Region','Body Style' ,'Company' , 'Holiday' ]:
  carSales[column] = label_encoder.fit_transform(carSales[column])

In [26]:
mean_map = carSales.groupby('Model')['Price ($)'].mean().to_dict()  
carSales['model'] = carSales['Model'].map(mean_map)
carSales.drop('Model', axis=1, inplace=True)

In [27]:
def show_unique_values(df):
    for col in df.select_dtypes(include=['object', 'category']).columns:
        print(f"Column: {col}")
        print(f"Unique Count: {df[col].nunique()}")
        print(f"Unique Percentage: {(df[col].nunique() / len(df)) * 100}%")
        print(f"the names of the unique values are: {df[col].unique()}")
        print("\n")
show_unique_values(carSales)

Column: Season
Unique Count: 4
Unique Percentage: 0.01732426696695396%
the names of the unique values are: ['Winter' 'Spring' 'Summer' 'Fall']




In [28]:
map_seaonal = {
    'Spring': 0,
    'Summer': 1,
    'Fall': 2,
    'Winter': 3
}
carSales['Season'] = carSales['Season'].map(map_seaonal)


In [29]:
carSales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23089 entries, 0 to 23089
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             23089 non-null  datetime64[ns]
 1   Gender           23089 non-null  int32         
 2   Annual Income    23089 non-null  int64         
 3   Company          23089 non-null  int32         
 4   Engine           23089 non-null  int32         
 5   Transmission     23089 non-null  int32         
 6   Color            23089 non-null  int32         
 7   Price ($)        23089 non-null  int64         
 8   Body Style       23089 non-null  int32         
 9   Dealer_Region    23089 non-null  int32         
 10  Year             23089 non-null  int32         
 11  Month            23089 non-null  int32         
 12  DayOfWeek        23089 non-null  int32         
 13  Season           23089 non-null  int64         
 14  Price_to_Income  23089 non-null  float64   

In [30]:
carSales.columns.all 

<bound method Index.all of Index(['Date', 'Gender', 'Annual Income', 'Company', 'Engine', 'Transmission',
       'Color', 'Price ($)', 'Body Style', 'Dealer_Region', 'Year', 'Month',
       'DayOfWeek', 'Season', 'Price_to_Income', 'Holiday', 'Is_Holiday',
       'Day', 'WeekOfYear', 'Is_Weekend', 'model'],
      dtype='object')>

In [31]:
carSales.head()

,Date,Gender,Annual Income,Company,Engine,Transmission,Color,Price ($),Body Style,Dealer_Region,...,Month,DayOfWeek,Season,Price_to_Income,Holiday,Is_Holiday,Day,WeekOfYear,Is_Weekend,model
0,2022-01-02,1,13500,8,0,0,0,26000,3,4,...,1,6,3,1.925926,11,0,2,52,1,22367.847619
1,2022-01-02,1,1480000,7,0,0,0,19000,3,0,...,1,6,3,0.012838,11,0,2,52,1,22465.016000
2,2022-01-02,1,1035000,4,1,1,2,31500,2,2,...,1,6,3,0.030435,11,0,2,52,1,42132.740909
3,2022-01-02,1,13500,27,1,1,1,14000,3,5,...,1,6,3,1.037037,11,0,2,52,1,26243.834356
4,2022-01-02,1,1465000,0,0,0,2,24500,1,3,...,1,6,3,0.016724,11,0,2,52,1,22819.456274


# **New Features Creation**

In [32]:
carSales["Engine_to_Model"] = carSales["Engine"] / (carSales["model"] + 1)

# Income Bracket using quartiles
carSales["Income_Bracket"] = pd.qcut(carSales["Annual Income"], q=4, labels=[1, 2, 3, 4])

# Seasonal Price Index
season_price_map = carSales.groupby('Season')["Price ($)"].mean().to_dict()
carSales["Seasonal_Price_Index"] = carSales["Season"].map(season_price_map)

# Year-Month Feature
carSales["Year_Month"] = carSales["Year"] * 100 + carSales["Month"]

# Engine × Transmission Combo
carSales["Engine_Transmission"] = carSales["Engine"].astype(str) + "_" + carSales["Transmission"].astype(str)
carSales["Engine_Transmission"] = LabelEncoder().fit_transform(carSales["Engine_Transmission"])

# Company Strength
company_avg_price = carSales.groupby("Company")["Price ($)"].mean().to_dict()
carSales["Company_Strength"] = carSales["Company"].map(company_avg_price)

In [33]:
carSales['Income_Bracket'].unique() 
carSales['Income_Bracket'].value_counts()

Income_Bracket
3    5796
1    5775
2    5775
4    5743
Name: count, dtype: int64

In [34]:
carSales["PI_plus_model"] = carSales["Price_to_Income"] + carSales["model"]
carSales["PI_plus_model"].head(10)


0    22369.773545
1    22465.028838
2    42132.771344
3    26244.871393
4    22819.472997
5    22098.387252
6    26632.348556
7    38749.086721
8    37202.718005
9    12870.741248
Name: PI_plus_model, dtype: float64

In [35]:

carSales['Income_Bracket'] = label_encoder.fit_transform(carSales['Income_Bracket'])


In [36]:
carSales = carSales.astype({col: 'int64' for col in carSales.select_dtypes(include='integer').columns})


In [37]:
carSales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23089 entries, 0 to 23089
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  23089 non-null  datetime64[ns]
 1   Gender                23089 non-null  int64         
 2   Annual Income         23089 non-null  int64         
 3   Company               23089 non-null  int64         
 4   Engine                23089 non-null  int64         
 5   Transmission          23089 non-null  int64         
 6   Color                 23089 non-null  int64         
 7   Price ($)             23089 non-null  int64         
 8   Body Style            23089 non-null  int64         
 9   Dealer_Region         23089 non-null  int64         
 10  Year                  23089 non-null  int64         
 11  Month                 23089 non-null  int64         
 12  DayOfWeek             23089 non-null  int64         
 13  Season               

In [38]:
check_data_types(carSales)

Column 'Date' has data type 'datetime64[ns]'


True

In [39]:
carSales.dtypes

Date                    datetime64[ns]
Gender                           int64
Annual Income                    int64
Company                          int64
Engine                           int64
Transmission                     int64
Color                            int64
Price ($)                        int64
Body Style                       int64
Dealer_Region                    int64
Year                             int64
Month                            int64
DayOfWeek                        int64
Season                           int64
Price_to_Income                float64
Holiday                          int64
Is_Holiday                       int64
Day                              int64
WeekOfYear                       int64
Is_Weekend                       int64
model                          float64
Engine_to_Model                float64
Income_Bracket                   int64
Seasonal_Price_Index           float64
Year_Month                       int64
Engine_Transmission      

In [40]:
scaler = StandardScaler()
carSales = carSales.copy()
carSales[['Annual Income', 'Price ($)']] = scaler.fit_transform(carSales[['Annual Income', 'Price ($)']])


#

## save 

In [41]:
joblib.dump(scaler, 'StanderScaler.pkl')
joblib.dump(label_encoder, 'LabelEncoder.pkl')

['LabelEncoder.pkl']

In [42]:
carSales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23089 entries, 0 to 23089
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  23089 non-null  datetime64[ns]
 1   Gender                23089 non-null  int64         
 2   Annual Income         23089 non-null  float64       
 3   Company               23089 non-null  int64         
 4   Engine                23089 non-null  int64         
 5   Transmission          23089 non-null  int64         
 6   Color                 23089 non-null  int64         
 7   Price ($)             23089 non-null  float64       
 8   Body Style            23089 non-null  int64         
 9   Dealer_Region         23089 non-null  int64         
 10  Year                  23089 non-null  int64         
 11  Month                 23089 non-null  int64         
 12  DayOfWeek             23089 non-null  int64         
 13  Season               

In [43]:
corr =carSales.corr()['Price ($)'].abs().sort_values(ascending=False)

# show the top 10 (skip the first one, which is Price itself)
top_feats = corr.iloc[1:11]
print(top_feats)
threshold = 0.2
top_feats = corr[corr > threshold].index.tolist()
print("Selected by corr-threshold:", top_feats)

PI_plus_model           0.493254
model                   0.493227
Price_to_Income         0.262996
Company_Strength        0.243971
Engine_to_Model         0.120120
Company                 0.022634
Body Style              0.014797
Holiday                 0.012945
Seasonal_Price_Index    0.010600
Engine                  0.010436
Name: Price ($), dtype: float64
Selected by corr-threshold: ['Price ($)', 'PI_plus_model', 'model', 'Price_to_Income', 'Company_Strength']


In [44]:
X = carSales.drop(columns=['Price ($)', 'Date'])
y = carSales['Price ($)']

In [45]:
# X, y as your full feature matrix & target
selector = SelectKBest(score_func=f_regression, k=5)
selector.fit(X, y)
selected = X.columns[selector.get_support()]
print("SelectKBest picked:", list(selected))


SelectKBest picked: ['Price_to_Income', 'model', 'Engine_to_Model', 'Company_Strength', 'PI_plus_model']


In [46]:
est = XGBRegressor(random_state=42)
rfe = RFE(estimator=est, n_features_to_select=5)
rfe.fit(X, y)
print("RFE selected:", list(X.columns[rfe.support_]))

RFE selected: ['Annual Income', 'Price_to_Income', 'model', 'Income_Bracket', 'PI_plus_model']


In [47]:
carSales.columns
#PI_plus_model           0.493254
#model                   0.493227
#Price_to_Income         0.262996
#Company_Strength        0.243971
#Engine_to_Model         0.120120
#Company                 0.022634
#Body Style              0.014797
#Holiday                 0.012945
#Seasonal_Price_Index    0.010600
#Engine                  0.010436
#Selected by corr-threshold: ['Price ($)', 'PI_plus_model', 'model', 'Price_to_Income', 'Company_Strength']
# ['Price_to_Income', 'model', 'Engine_to_Model', 'Company_Strength', 'PI_plus_model']
# ['Annual Income', 'Price_to_Income', 'model', 'Income_Bracket', 'PI_plus_model']

Index(['Date', 'Gender', 'Annual Income', 'Company', 'Engine', 'Transmission',
       'Color', 'Price ($)', 'Body Style', 'Dealer_Region', 'Year', 'Month',
       'DayOfWeek', 'Season', 'Price_to_Income', 'Holiday', 'Is_Holiday',
       'Day', 'WeekOfYear', 'Is_Weekend', 'model', 'Engine_to_Model',
       'Income_Bracket', 'Seasonal_Price_Index', 'Year_Month',
       'Engine_Transmission', 'Company_Strength', 'PI_plus_model'],
      dtype='object')

In [48]:
carSalesModel = carSales.drop(columns=[ 'Transmission', 'Color',  'Dealer_Region','Season' ,'Year', 'Month','Gender','DayOfWeek','Day','WeekOfYear','Year_Month','Engine_Transmission' ])

In [50]:
joblib.dump(carSales, '../Data/dataPreprocessing.pkl')
carSales.to_csv('../Data/dataPreprocessing.csv')
joblib.dump(carSalesModel, '../Data/dataModeling.pkl')
carSalesModel.to_csv('../Data/dataModeling.csv')